<a href="https://colab.research.google.com/github/ashutoshsinha25/pepcoding_FJP_DS/blob/main/Pytorch/FJP_DS_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
torch.__version__

'1.12.1+cu113'

In [2]:
a = torch.empty(6,3)
print(a)

tensor([[3.9891e-35, 0.0000e+00, 1.4293e-43],
        [0.0000e+00,        nan, 0.0000e+00],
        [1.1434e+27, 7.5555e+31, 1.2705e+31],
        [7.0948e+22, 3.8946e+21, 4.4650e+30],
        [1.1286e+27, 2.8405e+20, 3.2608e-12],
        [1.4583e-19, 1.0899e+27, 6.8943e+34]])


In [3]:
a = torch.rand(4,3)
print(a)

tensor([[0.8547, 0.8890, 0.2814],
        [0.5421, 0.1974, 0.1747],
        [0.1243, 0.8786, 0.5446],
        [0.4175, 0.0092, 0.6940]])


In [4]:
a = torch.zeros(4,3, dtype=torch.long)
print(a)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


In [5]:
a = torch.tensor([7.8, 5])
type(a)

torch.Tensor

In [6]:
a = a.new_ones(6,5, dtype=torch.double)    # new methods take in sizes
print(a)

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]], dtype=torch.float64)


In [7]:
print(a.size())

torch.Size([6, 5])


In [8]:
b = torch.rand(6,5)
print(a + b)

tensor([[1.1656, 1.2239, 1.9202, 1.5976, 1.8824],
        [1.6386, 1.7168, 1.7695, 1.5024, 1.1930],
        [1.4483, 1.7269, 1.2608, 1.7778, 1.1193],
        [1.1059, 1.3875, 1.0961, 1.2407, 1.8949],
        [1.4628, 1.8224, 1.6135, 1.3509, 1.6632],
        [1.8389, 1.2785, 1.9561, 1.7118, 1.7633]], dtype=torch.float64)


In [9]:
print(torch.add(a,b))

tensor([[1.1656, 1.2239, 1.9202, 1.5976, 1.8824],
        [1.6386, 1.7168, 1.7695, 1.5024, 1.1930],
        [1.4483, 1.7269, 1.2608, 1.7778, 1.1193],
        [1.1059, 1.3875, 1.0961, 1.2407, 1.8949],
        [1.4628, 1.8224, 1.6135, 1.3509, 1.6632],
        [1.8389, 1.2785, 1.9561, 1.7118, 1.7633]], dtype=torch.float64)


In [10]:
x = torch.ones(4)
print(x)

tensor([1., 1., 1., 1.])


In [11]:
y = x.numpy()
print(y)

[1. 1. 1. 1.]


In [12]:
import numpy as np
f = np.ones(4)
g = torch.from_numpy(f)
np.add(f, 4-7, out=f)
print(f)
print(g)

[-2. -2. -2. -2.]
tensor([-2., -2., -2., -2.], dtype=torch.float64)


In [13]:
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to("cuda")                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))

tensor([2., 2., 2., 2.], device='cuda:0')
tensor([2., 2., 2., 2.], dtype=torch.float64)


In [13]:
torch.cuda.is_available()

The key step is between the last convolution and the first Linear block. Conv2d outputs a tensor of shape [batch_size, n_features_conv, height, width] whereas Linear expects [batch_size, n_features_lin]. To make the two align you need to "stack" the 3 dimensions [n_features_conv, height, width] into one [n_features_lin]. As follows, it must be that n_features_lin == n_features_conv height width. In the original code this "stacking" is achieved by

x = x.view(-1, self.num_flat_features(x)) and if you inspect num_flat_features it just computes this n_features_conv height width product. In other words, your first conv must have num_flat_features(x) input features, where x is the tensor retrieved from the preceding convolution. But we need to calculate this value ahead of time, so that we can initialize the network in the first place...

The calculation follows from inspecting the operations one by one.

input is 32x32 we do a 5x5 convolution without padding, so we lose 2 pixels at each side, we drop down to 28x28 we do maxpooling with receptive field of 2x2, we cut each dimension by half, down to 14x14 we do another 5x5 convolution without padding, we drop down to 10x10 we do another maxpooling, we drop down to 5x5 and this 5x5 is why in the tutorial you see self.fc1 = nn.Linear(16 5 5, 120). It's n_features_conv height width, when starting from a 32x32 image. If you want to have a different input size, you have to redo the above calculation and adjust your first Linear layer accordingly.

For the further operations, it's just a chain of matrix multiplications (that's what Linear does). So the only rule is that the n_features_out of previous Linear matches n_features_in of the next one. Values 120 and 84 are entirely arbitrary, though they were probably chosen by the author such that the resulting network performs well.

In [14]:
#Lenet-5

import torch
import torch.nn as nn
import torch.nn.functional as F


class network(nn.Module):

    def __init__(self):
        super(network, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = mx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = network()
print(net)

network(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [15]:
list(net.parameters())[0]

Parameter containing:
tensor([[[[-0.0101, -0.1132,  0.2955],
          [ 0.1036,  0.2889, -0.0141],
          [-0.0394, -0.3024,  0.1244]]],


        [[[ 0.0611, -0.1684, -0.2600],
          [ 0.2948,  0.2902,  0.1060],
          [ 0.2577,  0.2920, -0.1733]]],


        [[[ 0.1076,  0.3089, -0.1817],
          [-0.0684,  0.0704, -0.0017],
          [ 0.2240, -0.2279, -0.2678]]],


        [[[ 0.1133, -0.2980,  0.1241],
          [-0.0540, -0.1369, -0.3018],
          [ 0.1799, -0.0231, -0.0166]]],


        [[[-0.1684,  0.1903, -0.3323],
          [ 0.2317, -0.0838,  0.2172],
          [-0.0254, -0.2626, -0.1478]]],


        [[[-0.1350, -0.0932,  0.0233],
          [ 0.0839,  0.1601,  0.1253],
          [ 0.1746, -0.2013, -0.0059]]]], requires_grad=True)

In [16]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [17]:
inp = torch.randn(1, 1, 32, 32)
out = net(inp)
print(out)

tensor([[ 0.0244, -0.0217,  0.0061, -0.0161,  0.0589, -0.1095,  0.0262,  0.0180,
          0.0731, -0.0746]], grad_fn=<AddmmBackward0>)


In [18]:
output = net(inp)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.4002, grad_fn=<MseLossBackward0>)


In [19]:
torch.randn(1, 1, 32, 32)

tensor([[[[ 1.2674, -0.7316,  1.0984,  ...,  1.0552,  0.6202,  1.4816],
          [-0.3044, -0.3760,  0.8253,  ..., -1.2952, -2.1812,  0.4777],
          [ 1.0150, -1.9161,  1.1320,  ..., -2.1898, -0.1256,  1.3111],
          ...,
          [-0.3199,  0.4530, -1.0230,  ..., -0.7003, -0.5108,  0.8114],
          [-0.4863, -0.1831,  0.9184,  ...,  1.5777,  0.9629,  0.8461],
          [ 0.2601, -0.6687, -0.2829,  ...,  0.5204,  0.5916, -1.1453]]]])